In [12]:
import json
import requests


def chat_completion(
    prompt,
    system_message="You are a helpful coding assistant.",
    max_tokens=512,
    temperature=0.01,
):
    # Replace with your actual localtunnel URL
    LOCALTUNNEL_URL = "https://yummy-buses-behave.loca.lt"

    response = requests.post(
        f"{LOCALTUNNEL_URL}/chat/completions",
        headers={"Content-Type": "application/json"},
        json={
            "messages": [
                {"role": "system", "content": system_message},
                {"role": "user", "content": prompt},
            ],
            "max_new_tokens": max_tokens,
            "temperature": temperature,
        },
    )

    return response.json()


# Example usage
result = chat_completion("Write a Python function to find prime numbers up to n.")
print(json.dumps(result, indent=2))


{
  "generated_text": "Here is a simple Python function that uses the Sieve of Eratosthenes algorithm to find all prime numbers up to n:\n\n```python\ndef sieve_of_eratosthenes(n):\n    primes = [True for i in range(n+1)]\n    p = 2\n    while (p * p <= n):\n        if (primes[p] == True):\n            for i in range(p * p, n+1, p):\n                primes[i] = False\n        p += 1\n    prime_numbers = []\n    for p in range(2, n):\n        if primes[p]:\n            prime_numbers.append(p)\n    return prime_numbers\n\n# Test the function\nprint(sieve_of_eratosthenes(30))\n```\n\nThis function first initializes a list of boolean values representing whether each number is prime or not. It then iterates over each number starting from 2 and marks its multiples as non-prime. Finally, it returns a list of all prime numbers up to n.\n\nThe output of this script will be: `[2, 3, 5, 7, 11, 13, 17, 19, 23, 29]` which are all prime numbers less than or equal to 30."
}


In [17]:
instructions = """You are a coding assistant that help to answer multiple choices of software development and Extract final answer. \
                    The answer selected from the multiple choice options. Should be one alphabet option (A,B,C,D,E,F)
                """
FEW_SHOT_TEMPLATE = """
The following are multiple choice questions (with answers) about software development.

Question: If a sorted array of integers is guaranteed to not contain duplicate values, in order to search for a specific value which of the following algorithms is the most efficient for this task?
(A) Bubble Sort
(B) Linear Search
(C) Insertion Sort
(D) Binary Search

Answer: {{"answer": "D"}}

Question: {question}
{multiple_choices}

Answer in JSON format:
```json
{{"answer": "Your selected option here"}}
``` 
"""
question = """
Question: What will be output of the following c code?
#include<stdio.h>
int main()
{
    int a= sizeof(signed) +sizeof(unsigned);
    int b=sizeof(const)+sizeof(volatile);
    printf(""%d"",a+++b);
    return 0;
}
"""
choices = """
['A: 10', 'B: 9', 'C: 8', 'D: Error']
"""
prompt = FEW_SHOT_TEMPLATE.format(question=question, multiple_choices=choices)
output_json = chat_completion(prompt=prompt, system_message=instructions)

In [21]:
output_json["generated_text"]


'```json\n{"answer": "D"}\n```'